In [ ]:
from gtts import gTTS
from IPython.display import Audio, display
import sqlite3
import pandas as pd

def speak_arabic(text):
    tts = gTTS(text=text, lang='ar', slow=False)
    tts.save("output.mp3")
    display(Audio("output.mp3", autoplay=True))

def init_database():
    conn = sqlite3.connect('hajj_bot.db')
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS hajj_companies (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            company_name_ar TEXT NOT NULL,
            company_name_en TEXT,
            city TEXT,
            country TEXT,
            email TEXT,
            contact_info TEXT,
            is_authorized BOOLEAN NOT NULL
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS queries (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_query TEXT NOT NULL,
            bot_response TEXT NOT NULL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')

    conn.commit()
    conn.close()

def load_excel_to_db(filename):
    df = pd.read_excel(filename)
    conn = sqlite3.connect('hajj_bot.db')

    for _, row in df.iterrows():
        conn.execute('''
            INSERT INTO hajj_companies
            (company_name_ar, company_name_en, city, country, email, contact_info, is_authorized)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (row['Hajj Company (Arabic)'], row['Hajj Company (English)'],
              row['City'], row['Country'], row['email'], row['Contact_Info'],
              1 if str(row['is_autorized']).lower() in ['true', '1', 'yes'] else 0))

    conn.commit()
    conn.close()

def check_hajj_company(company_name):
    conn = sqlite3.connect('hajj_bot.db')
    cursor = conn.cursor()

    cursor.execute('''
        SELECT company_name_ar, city, country, email, contact_info, is_authorized
        FROM hajj_companies
        WHERE company_name_ar LIKE ? OR company_name_en LIKE ?
    ''', (f'%{company_name}%', f'%{company_name}%'))

    result = cursor.fetchone()

    if result:
        name, city, country, email, contact, is_auth = result
        if is_auth:
            response = f"Company {name} is authorized. City: {city}, Contact: {contact}"
        else:
            response = f"Warning! Company {name} is NOT authorized"
    else:
        response = f"No information found for {company_name}"

    cursor.execute('INSERT INTO queries (user_query, bot_response) VALUES (?, ?)',
                   (company_name, response))
    conn.commit()
    conn.close()

    return response

def hajj_voice_bot(company_name):
    response = check_hajj_company(company_name)
    print(response)
    speak_arabic(response.split('.')[0])
    return response

init_database()

In [ ]:
!pip install gtts pandas openpyxl -q

from gtts import gTTS
from IPython.display import Audio, display
import sqlite3
import pandas as pd
from google.colab import files
import os

if os.path.exists('hajj_bot.db'):
    os.remove('hajj_bot.db')

uploaded = files.upload()
excel_file = list(uploaded.keys())[0]

df = pd.read_excel(excel_file)
print(f"Loaded {len(df)} rows from Excel")

conn = sqlite3.connect('hajj_bot.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE hajj_companies (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    company_name_ar TEXT,
    company_name_en TEXT,
    city TEXT,
    country TEXT,
    email TEXT,
    contact_info TEXT,
    is_authorized BOOLEAN)''')

cursor.execute('''CREATE TABLE queries (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_query TEXT,
    bot_response TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP)''')

inserted = 0
for _, row in df.iterrows():
    name_ar = str(row['Hajj Company (Arabic)']).strip() if pd.notna(row['Hajj Company (Arabic)']) else None
    name_en = str(row['Hajj Company (English)']).strip() if pd.notna(row['Hajj Company (English)']) else None

    if not name_ar and not name_en:
        continue

    city = str(row['City']).strip() if pd.notna(row['City']) else None
    country = str(row['Country']).strip() if pd.notna(row['Country']) else None
    email = str(row['email']).strip() if pd.notna(row['email']) else None
    contact = str(row['Contact_Info']).strip() if pd.notna(row['Contact_Info']) else None
    is_auth = 1 if str(row['is_autorized']).lower() in ['true', '1', 'yes'] else 0

    cursor.execute('''INSERT INTO hajj_companies
        (company_name_ar, company_name_en, city, country, email, contact_info, is_authorized)
        VALUES (?, ?, ?, ?, ?, ?, ?)''',
        (name_ar, name_en, city, country, email, contact, is_auth))
    inserted += 1

conn.commit()
conn.close()
print(f"Inserted {inserted} companies into database")

def speak_arabic(text):
    tts = gTTS(text=text, lang='ar', slow=False)
    tts.save("output.mp3")
    display(Audio("output.mp3", autoplay=True))

def hajj_voice_bot(company_name):
    conn = sqlite3.connect('hajj_bot.db')
    cursor = conn.cursor()

    cursor.execute('''SELECT company_name_ar, company_name_en, city, is_authorized
        FROM hajj_companies
        WHERE company_name_ar LIKE ? OR company_name_en LIKE ?''',
        (f'%{company_name}%', f'%{company_name}%'))

    result = cursor.fetchone()

    if result:
        name_ar, name_en, city, is_auth = result
        name = name_ar if name_ar else name_en
        if is_auth:
            response = f"{name} is authorized"
            if city:
                response += f" in {city}"
        else:
            response = f"Warning: {name} is NOT authorized"
    else:
        response = f"Company {company_name} not found"

    cursor.execute('INSERT INTO queries (user_query, bot_response) VALUES (?, ?)',
                   (company_name, response))
    conn.commit()
    conn.close()

    print(response)
    speak_arabic(response)
    return response

def show_stats():
    conn = sqlite3.connect('hajj_bot.db')
    cursor = conn.cursor()

    total = cursor.execute('SELECT COUNT(*) FROM hajj_companies').fetchone()[0]
    authorized = cursor.execute('SELECT COUNT(*) FROM hajj_companies WHERE is_authorized = 1').fetchone()[0]

    conn.close()

    print(f"\nTotal companies: {total}")
    print(f"Authorized: {authorized}")
    print(f"Unauthorized: {total - authorized}")

speak_arabic("Welcome to Hajj Bot")
show_stats()

Saving Hajj_Info_Final_filled (3).xlsx to Hajj_Info_Final_filled (3) (3).xlsx
Loaded 8345 rows from Excel
Inserted 8345 companies into database



Total companies: 8345
Authorized: 8282
Unauthorized: 63


In [ ]:
!pip install gtts pandas openpyxl -q

from gtts import gTTS
from IPython.display import Audio, display, HTML
import sqlite3
import pandas as pd
import ipywidgets as widgets

def speak_arabic(text):
    tts = gTTS(text=text, lang='ar', slow=False)
    tts.save("response.mp3")
    display(Audio("response.mp3", autoplay=True))

def check_company_voice(company_name):
    conn = sqlite3.connect('hajj_bot.db')
    cursor = conn.cursor()

    cursor.execute('''
        SELECT company_name_ar, company_name_en, city, country,
               contact_info, is_authorized
        FROM hajj_companies
        WHERE company_name_ar LIKE ? OR company_name_en LIKE ?
    ''', (f'%{company_name}%', f'%{company_name}%'))

    result = cursor.fetchone()

    print("\n" + "="*70)
    print(f"🕋 استفسار: هل شركة {company_name} مرخصة؟")
    print("="*70)

    if result:
        name_ar, name_en, city, country, contact, is_auth = result
        name = name_ar if name_ar else name_en

        if is_auth:
            response = f"نعم، شركة {name} مرخصة من وزارة الحج والعمرة"
            if city:
                response += f" في مدينة {city}"
            if contact:
                response += f". للتواصل: {contact}"

            print(f"\n✅ {response}\n")

        else:
            response = f"تحذير! شركة {name} غير مرخصة من وزارة الحج والعمرة"
            print(f"\n⚠️ {response}\n")
    else:
        response = f"عذراً، لم أجد معلومات عن شركة {company_name} في قاعدة البيانات"
        print(f"\n❌ {response}\n")

    cursor.execute('INSERT INTO queries (user_query, bot_response) VALUES (?, ?)',
                   (company_name, response))
    conn.commit()
    conn.close()

    print("🔊 تشغيل الرد الصوتي...")
    print("="*70 + "\n")
    speak_arabic(response)

    return response

display(HTML("""
<style>
    .chat-container {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        border-radius: 15px;
        padding: 30px;
        color: white;
        text-align: center;
        margin: 20px 0;
    }
</style>
<div class="chat-container">
    <h1>🕋 بوت الحج الصوتي</h1>
    <h3>Hajj Company Verification Voice Bot</h3>
    <p>تحقق من ترخيص شركات الحج صوتياً</p>
</div>
"""))

text_input = widgets.Text(
    placeholder='مثال: جبل عمر، الطيبات، النور',
    description='اسم الشركة:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='600px')
)

button = widgets.Button(
    description='🔍 تحقق',
    button_style='success',
    layout=widgets.Layout(width='120px', height='40px')
)

voice_button = widgets.Button(
    description='🔊 تحقق + صوت',
    button_style='info',
    layout=widgets.Layout(width='150px', height='40px')
)

output = widgets.Output()

def on_check_click(b):
    with output:
        output.clear_output()
        if text_input.value:
            check_company_voice(text_input.value)
        else:
            print("⚠️ الرجاء إدخال اسم الشركة")

button.on_click(on_check_click)
voice_button.on_click(on_check_click)

display(text_input)
display(widgets.HBox([button, voice_button]))
display(output)

print("\n💡 أو استخدم الأمر مباشرة:")
print("   check_company_voice('جبل عمر')")

Text(value='', description='اسم الشركة:', layout=Layout(width='600px'), placeholder='مثال: جبل عمر، الطيبات، ا…

Output()


💡 أو استخدم الأمر مباشرة:
   check_company_voice('جبل عمر')


In [ ]:
!pip install gtts pandas openpyxl langdetect -q

from gtts import gTTS
from IPython.display import Audio, display, HTML
import sqlite3
import pandas as pd
import ipywidgets as widgets
from langdetect import detect

LANGUAGES = {
    'ar': {'name': 'العربية', 'code': 'ar'},
    'en': {'name': 'English', 'code': 'en'},
    'ur': {'name': 'اردو', 'code': 'ur'},
    'tr': {'name': 'Türkçe', 'code': 'tr'},
    'id': {'name': 'Indonesian', 'code': 'id'},
    'bn': {'name': 'বাংলা', 'code': 'bn'},
    'fr': {'name': 'Français', 'code': 'fr'},
    'es': {'name': 'Español', 'code': 'es'},
    'hi': {'name': 'हिन्दी', 'code': 'hi'},
    'ml': {'name': 'മലയാളം', 'code': 'ml'},
}

def detect_language(text):
    try:
        lang = detect(text)
        return lang if lang in LANGUAGES else 'en'
    except:
        return 'en'

def speak_multilingual(text, lang='auto'):
    if lang == 'auto':
        lang = detect_language(text)

    tts = gTTS(text=text, lang=lang, slow=False)
    tts.save("response.mp3")
    display(Audio("response.mp3", autoplay=True))

def translate_response(company_name, city, contact, is_authorized, lang):
    responses = {
        'ar': {
            'authorized': f"نعم، شركة {company_name} مرخصة من وزارة الحج والعمرة",
            'city': f" في مدينة {city}",
            'contact': f". للتواصل: {contact}",
            'not_authorized': f"تحذير! شركة {company_name} غير مرخصة من وزارة الحج والعمرة",
            'not_found': f"عذراً، لم أجد معلومات عن شركة {company_name}",
            'ministry': "الرجاء التواصل مع وزارة الحج: 920002814"
        },
        'en': {
            'authorized': f"Yes, {company_name} is authorized by the Ministry of Hajj and Umrah",
            'city': f" in {city}",
            'contact': f". Contact: {contact}",
            'not_authorized': f"Warning! {company_name} is NOT authorized by the Ministry of Hajj and Umrah",
            'not_found': f"Sorry, no information found for {company_name}",
            'ministry': "Please contact Ministry of Hajj: 920002814"
        },
        'ur': {
            'authorized': f"جی ہاں، {company_name} وزارت حج اور عمرہ سے مجاز ہے",
            'city': f" {city} میں",
            'contact': f". رابطہ: {contact}",
            'not_authorized': f"انتباہ! {company_name} وزارت حج اور عمرہ سے مجاز نہیں ہے",
            'not_found': f"معذرت، {company_name} کے بارے میں کوئی معلومات نہیں ملی",
            'ministry': "براہ کرم وزارت حج سے رابطہ کریں: 920002814"
        },
        'tr': {
            'authorized': f"Evet, {company_name} Hac ve Umre Bakanlığı tarafından yetkilendirilmiştir",
            'city': f" {city} şehrinde",
            'contact': f". İletişim: {contact}",
            'not_authorized': f"Uyarı! {company_name} Hac ve Umre Bakanlığı tarafından yetkilendirilmemiştir",
            'not_found': f"Üzgünüz, {company_name} hakkında bilgi bulunamadı",
            'ministry': "Lütfen Hac Bakanlığı ile iletişime geçin: 920002814"
        },
        'id': {
            'authorized': f"Ya, {company_name} diotorisasi oleh Kementerian Haji dan Umrah",
            'city': f" di kota {city}",
            'contact': f". Kontak: {contact}",
            'not_authorized': f"Peringatan! {company_name} TIDAK diotorisasi oleh Kementerian Haji dan Umrah",
            'not_found': f"Maaf, tidak ada informasi tentang {company_name}",
            'ministry': "Silakan hubungi Kementerian Haji: 920002814"
        },
        'bn': {
            'authorized': f"হ্যাঁ, {company_name} হজ ও ওমরাহ মন্ত্রণালয় দ্বারা অনুমোদিত",
            'city': f" {city} শহরে",
            'contact': f". যোগাযোগ: {contact}",
            'not_authorized': f"সতর্কতা! {company_name} হজ ও ওমরাহ মন্ত্রণালয় দ্বারা অনুমোদিত নয়",
            'not_found': f"দুঃখিত, {company_name} সম্পর্কে কোন তথ্য পাওয়া যায়নি",
            'ministry': "দয়া করে হজ মন্ত্রণালয়ের সাথে যোগাযোগ করুন: 920002814"
        },
        'fr': {
            'authorized': f"Oui, {company_name} est autorisée par le Ministère du Hajj et de la Omra",
            'city': f" à {city}",
            'contact': f". Contact: {contact}",
            'not_authorized': f"Attention! {company_name} N'EST PAS autorisée par le Ministère du Hajj",
            'not_found': f"Désolé, aucune information trouvée pour {company_name}",
            'ministry': "Veuillez contacter le Ministère du Hajj: 920002814"
        },
        'es': {
            'authorized': f"Sí, {company_name} está autorizada por el Ministerio de Hajj y Umrah",
            'city': f" en {city}",
            'contact': f". Contacto: {contact}",
            'not_authorized': f"¡Advertencia! {company_name} NO está autorizada por el Ministerio de Hajj",
            'not_found': f"Lo siento, no se encontró información sobre {company_name}",
            'ministry': "Por favor contacte al Ministerio de Hajj: 920002814"
        },
        'hi': {
            'authorized': f"हां, {company_name} हज और उमराह मंत्रालय द्वारा अधिकृत है",
            'city': f" {city} में",
            'contact': f". संपर्क: {contact}",
            'not_authorized': f"चेतावनी! {company_name} हज और उमराह मंत्रालय द्वारा अधिकृत नहीं है",
            'not_found': f"क्षमा करें, {company_name} के बारे में कोई जानकारी नहीं मिली",
            'ministry': "कृपया हज मंत्रालय से संपर्क करें: 920002814"
        },
        'ml': {
            'authorized': f"അതെ, {company_name} ഹജ്ജ് ഉംറ മന്ത്രാലയം അംഗീകരിച്ചതാണ്",
            'city': f" {city} നഗരത്തിൽ",
            'contact': f". ബന്ധപ്പെടുക: {contact}",
            'not_authorized': f"മുന്നറിയിപ്പ്! {company_name} ഹജ്ജ് മന്ത്രാലയം അംഗീകരിച്ചതല്ല",
            'not_found': f"ക്ഷമിക്കണം, {company_name} കുറിച്ച് വിവരങ്ങളൊന്നും കണ്ടെത്തിയില്ല",
            'ministry': "ഹജ്ജ് മന്ത്രാലയവുമായി ബന്ധപ്പെടുക: 920002814"
        }
    }

    if lang not in responses:
        lang = 'en'

    if is_authorized:
        response = responses[lang]['authorized']
        if city:
            response += responses[lang]['city']
        if contact:
            response += responses[lang]['contact']
    else:
        response = responses[lang]['not_authorized']
        response += ". " + responses[lang]['ministry']

    return response

def check_company_multilingual(company_name, language='auto'):
    if language == 'auto':
        language = detect_language(company_name)

    conn = sqlite3.connect('hajj_bot.db')
    cursor = conn.cursor()

    cursor.execute('''
        SELECT company_name_ar, company_name_en, city, country,
               contact_info, is_authorized
        FROM hajj_companies
        WHERE company_name_ar LIKE ? OR company_name_en LIKE ?
    ''', (f'%{company_name}%', f'%{company_name}%'))

    result = cursor.fetchone()

    print("\n" + "="*70)

    if result:
        name_ar, name_en, city, country, contact, is_auth = result
        name = name_ar if name_ar else name_en

        response = translate_response(name, city, contact, is_auth, language)

        if is_auth:
            print(f"✅ {response}\n")
        else:
            print(f"⚠️ {response}\n")
    else:
        if language == 'ar':
            response = f"عذراً، لم أجد معلومات عن شركة {company_name}"
        elif language == 'ur':
            response = f"معذرت، {company_name} کے بارے میں کوئی معلومات نہیں ملی"
        else:
            response = f"Sorry, no information found for {company_name}"

        print(f"❌ {response}\n")

    cursor.execute('INSERT INTO queries (user_query, bot_response) VALUES (?, ?)',
                   (company_name, response))
    conn.commit()
    conn.close()

    print(f"🔊 Playing response in {LANGUAGES.get(language, {}).get('name', language)}...")
    print("="*70 + "\n")
    speak_multilingual(response, language)

    return response

display(HTML("""
<style>
    .chat-container {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        border-radius: 15px;
        padding: 30px;
        color: white;
        text-align: center;
        margin: 20px 0;
    }
</style>
<div class="chat-container">
    <h1>🕋 🌍 Multilingual Hajj Voice Bot</h1>
    <h3>بوت الحج متعدد اللغات</h3>
    <p>Supports: العربية • English • اردو • Türkçe • Indonesian • বাংলা • Français • Español • हिन्दी • മലയാളം</p>
</div>
"""))

text_input = widgets.Text(
    placeholder='Example: جبل عمر, Jabal Omar, etc.',
    description='Company:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='600px')
)

language_dropdown = widgets.Dropdown(
    options=[('Auto Detect', 'auto')] + [(v['name'], k) for k, v in LANGUAGES.items()],
    value='auto',
    description='Language:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='300px')
)

button = widgets.Button(
    description='🔍 Check',
    button_style='success',
    layout=widgets.Layout(width='120px', height='40px')
)

voice_button = widgets.Button(
    description='🔊 Check + Voice',
    button_style='info',
    layout=widgets.Layout(width='150px', height='40px')
)

output = widgets.Output()

def on_check_click(b):
    with output:
        output.clear_output()
        if text_input.value:
            check_company_multilingual(text_input.value, language_dropdown.value)
        else:
            print("⚠️ Please enter company name")

button.on_click(on_check_click)
voice_button.on_click(on_check_click)

display(text_input)
display(language_dropdown)
display(widgets.HBox([button, voice_button]))
display(output)

print("\n💡 Direct usage:")
print("   check_company_multilingual('جبل عمر', 'ar')")
print("   check_company_multilingual('Jabal Omar', 'en')")
print("   check_company_multilingual('جبل عمر', 'ur')")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Text(value='', description='Company:', layout=Layout(width='600px'), placeholder='Example: جبل عمر, Jabal Omar…

Dropdown(description='Language:', layout=Layout(width='300px'), options=(('Auto Detect', 'auto'), ('العربية', …

Output()


💡 Direct usage:
   check_company_multilingual('جبل عمر', 'ar')
   check_company_multilingual('Jabal Omar', 'en')
   check_company_multilingual('جبل عمر', 'ur')
